# [LangChain](https://docs.langchain.com/docs/)

<br>

- LangChain is a framework for developing applications powered by language models.

## Topics Covered

1. Vectorstore Retrieval
2. Chatbot

## Installation

```sh
pip install langchain

# OR
pip install 'langchain[all]'

# Other dependencies
pip install python-dotenv
pip install openai
```

In [1]:
# Built-in library
import itertools
import re
import json
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pprint import pprint
import pandas as pd

# Visualization
import matplotlib.pyplot as plt


# pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black
# auto reload imports
%load_ext autoreload
%autoreload 2

## 1. Vectorstore Retrieval

```text

Retrieval
---------
- This is the process of finding documents that are relevant to a given query. 
- It's done by using a semantic search algorithm to calculate the similarity between the query and the documents in a corpus. 
- The documents that are most similar to the query are then retrieved.

- It's an important part of NLP chatbot pipeline because it allows the system to access information that's not already stored in the language model. 
- It can be useful for a variety of tasks, such as answering questions, generating text, and translating languages.
```

<br>

```sh
# Install dependency
pip install lark
```


In [2]:
# Similarity Search
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


persist_directory = "../../data/chroma/"
# Create embeddings
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

print(vectordb._collection.count())

418


### 1. Document Similarity

- Semantic search algorithm

In [3]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]
# Create DB
smalldb = Chroma.from_texts(texts, embedding=embedding)
question = "Tell me about all-white mushrooms with large fruiting bodies"

# Find similar documents
# This does NOT select docs that are diverse and contain important info.
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).', metadata={})]

### 2. Maximum Marginal Relevance

```text
- This returns docs selected using the maximal marginal relevance. 
- Maximal marginal relevance optimizes for `similarity to query` AND `diversity among selected documents`.
```

In [4]:
smalldb.max_marginal_relevance_search(
    question,
    k=2,  # Num of Documents to return
    fetch_k=3,  # Num of Documents to fetch to pass to MMR algorithm.
)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.', metadata={})]

In [5]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question, k=3)

# Notice that the results are exactly the same
# i.e. they are duplicated
pprint(docs_ss[0].page_content[:100])
pprint(docs_ss[1].page_content[:100])

('those homeworks will be done in either MATLA B or in Octave, which is sort '
 'of — I \n'
 'know some people ')
('those homeworks will be done in either MATLA B or in Octave, which is sort '
 'of — I \n'
 'know some people ')


In [6]:
# Compare the difference in results with MMR.
docs_mmr = vectordb.max_marginal_relevance_search(question, k=3)

# Notice that the results are different!
pprint(docs_mmr[0].page_content[:100])
pprint(docs_mmr[1].page_content[:100])

('those homeworks will be done in either MATLA B or in Octave, which is sort '
 'of — I \n'
 'know some people ')
('into his office and he said, "Oh, professo r, professor, thank you so much '
 'for your \n'
 'machine learnin')


### 3a. Addressing Specificity: Working with metadata

```text
- There are times when the search results include results from other documents that are NOT relevant.
- To address this, many vectorstores support operations on metadata.
- Metadata provides context for each embedded chunk.
```

In [7]:
fp = "../../data/cs229-data/MachineLearning-Lecture03.pdf"
question = "what did they say about regression in the third lecture?"

# Search for similar docs using the metadata which provides more context.
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source": fp},  # Ensures that this document is searched
)
for d in docs:
    print(d.metadata)

{'page': 0, 'source': '../../data/cs229-data/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': '../../data/cs229-data/MachineLearning-Lecture03.pdf'}
{'page': 14, 'source': '../../data/cs229-data/MachineLearning-Lecture03.pdf'}


### 3b. Inferring The Metadata Using An LLM

```text
Addressing Specificity: working with metadata using self-query retriever.
- There's an interesting challenge: we often want to infer the metadata from the query itself.
- To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
- The query string to use for vector search
  - A metadata filter to pass in as well.
- Most vector databases support metadata filters, so this doesn't require any new databases or indexes.
```

In [31]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo


fp = "../../data/cs229-data/MachineLearning-Lecture"
DESCRIPTION = f"The lecture the chunk is from, should be one of `{fp}01.pdf`, `{fp}02.pdf`, or `{fp}03.pdf`"

METADATA_FIELD_INFO = [
    AttributeInfo(
        name="source",
        description=DESCRIPTION,
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

METADATA_FIELD_INFO

[AttributeInfo(name='source', description='The lecture the chunk is from, should be one of `../../data/cs229-data/MachineLearning-Lecture01.pdf`, `../../data/cs229-data/MachineLearning-Lecture02.pdf`, or `../../data/cs229-data/MachineLearning-Lecture03.pdf`', type='string'),
 AttributeInfo(name='page', description='The page from the lecture', type='integer')]

In [32]:
DOCUMENT_CONTENT_DESCRIPTION = "Lecture notes"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectordb, DOCUMENT_CONTENT_DESCRIPTION, METADATA_FIELD_INFO, verbose=True
)
question = "what did they say about regression in the third lecture?"

docs = retriever.get_relevant_documents(question)

query='regression' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='source', value='../../data/cs229-data/MachineLearning-Lecture03.pdf') limit=None


In [33]:
# You can see that all the query results are from lecture 3!
for d in docs:
    print(d.metadata)

{'page': 14, 'source': '../../data/cs229-data/MachineLearning-Lecture03.pdf'}
{'page': 14, 'source': '../../data/cs229-data/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': '../../data/cs229-data/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': '../../data/cs229-data/MachineLearning-Lecture03.pdf'}


In [43]:
# You can see that all the query results are from lecture 3!
for d in docs:
    print(d.page_content[:200])
    print("\n", "======" * 20)

Student: It’s the lowest it –  
Instructor (Andrew Ng) :No, exactly. Right. So zero to the same, this is not the same, 
right? And the reason is, in logi stic regression this is diffe rent from before

Student: It’s the lowest it –  
Instructor (Andrew Ng) :No, exactly. Right. So zero to the same, this is not the same, 
right? And the reason is, in logi stic regression this is diffe rent from before

MachineLearning-Lecture03  
Instructor (Andrew Ng) :Okay. Good morning and welcome b ack to the third lecture of 
this class. So here’s what I want to do t oday, and some of the topics I do today may 

MachineLearning-Lecture03  
Instructor (Andrew Ng) :Okay. Good morning and welcome b ack to the third lecture of 
this class. So here’s what I want to do t oday, and some of the topics I do today may 



In [30]:
smalldb.similarity_search(question, k=2)

[]

### 4. Compression

```text
- Compression is another approach for improving the quality of retrieved docs.
- Information most relevant to a query may be buried in a document with a lot of irrelevant text.
- Passing that full document through your application can lead to more expensive LLM calls and poorer responses.
- Contextual compression is meant to fix this.
```

In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [11]:
llm = OpenAI(temperature=0)

# DocumentCompressor that uses an LLM chain to extract the relevant parts of documents.
compressor = LLMChainExtractor.from_llm(llm)

# Retriever that compresses the result
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type="mmr"),
)

question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 2:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 3:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's so

### 5. Other types of retrieval

```text
- The LangChain retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.
```


In [15]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Load PDF
fp = "../../data/cs229-data/MachineLearning-Lecture01.pdf"
loader = PyPDFLoader(file_path=fp)
pages = loader.load()

# Extract all the text from the pages
all_page_text = [p.page_content for p in pages]
joined_page_text = " ".join(all_page_text)
joined_page_text[:100]

'MachineLearning-Lecture01  \nInstructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machi'

In [16]:
# Split the docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_text(joined_page_text)

# Retrieve the info using SVM
svm_retriever = SVMRetriever.from_texts(splits, embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

question = "What are major topics for this class?"
docs_svm = svm_retriever.get_relevant_documents(question)
docs_svm[0]

Document(page_content="let me just check what questions you have righ t now. So if there are no questions, I'll just \nclose with two reminders, which are after class today or as you start to talk with other \npeople in this class, I just encourage you again to start to form project partners, to try to \nfind project partners to do your project with. And also, this is a good time to start forming \nstudy groups, so either talk to your friends  or post in the newsgroup, but we just \nencourage you to try to star t to do both of those today, okay? Form study groups, and try \nto find two other project partners.  \nSo thank you. I'm looking forward to teaching this class, and I'll see you in a couple of \ndays.   [End of Audio]  \nDuration: 69 minutes", metadata={})

In [19]:
# Retrieve the info using TFIDF
question = "what did they say about matlab?"
docs_tfidf = tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is the first ste

<br><hr>

## Chatbot

- [Example notebook](https://github.com/pinecone-io/examples/blob/master/generation/langchain/handbook/05-langchain-retrieval-augmentation.ipynb)
  
<br>

### Querying A Vectorstore

<br>

[![image.png](https://i.postimg.cc/7PWxf0dh/image.png)](https://postimg.cc/YjQcPGpB)

<br>

```text
1. Using the embedding model create vector embeddings for the context to be indexed.
2. Insert the veector embeddings into the vector DB with some reference to the original context the embeddings were created from.
3. When a query is made using the application, embed the query using the same embedding model and query the vector DB for similar vector embeddings.
```

In [20]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


persist_directory = "../../data/chroma/"
embedding = OpenAIEmbeddings()

# Load the embedded data
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

question = "What are major topics for this class?"
docs = vectordb.similarity_search(question, k=3)
len(docs)

3

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0)


# Build prompt
template = """Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know, don't try to \
make up an answer. Use three sentences maximum. Keep the answer as concise as possible. \
Always say "thanks for asking!" at the end of the answer. 

{context}

Question: {question}

Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Run chain
question = "Is probability a class topic?"
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic. Thanks for asking!'

### Add Memory

In [23]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


# Create memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ConversationalRetrievalChain
retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)
question = "Is probability a class topic?"
result = qa({"question": question})

result["answer"]

'Yes, probability is a topic that will be covered in this class. The instructor assumes that students have familiarity with basic probability and statistics.'

In [26]:
question = "Who taught the class?"
result = qa({"question": question})

result["answer"]

'The instructor of the class was Andrew Ng.'

In [27]:
question = "what did they say about matlab?"
result = qa({"question": question})

result["answer"]

'MATLAB is described as a programming language that makes it easy to write codes using matrices, perform numerical routines, manipulate data, and plot data. It is also mentioned that MATLAB is an easy-to-learn tool for implementing learning algorithms. Additionally, it is mentioned that MATLAB can be used for the homework assignments in the class. It is also mentioned that Octave is a software package that is similar to MATLAB and can be downloaded for free off the internet. Octave has fewer features than MATLAB but can be used for most purposes in the class.'